<a href="https://colab.research.google.com/github/he9180/FISTA-lasso/blob/main/Copy_of_Testing_IRG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Numerical experiments for IRG methods. First we import some necessary packages.

In [ ]:
from scipy.optimize import rosen
from scipy.optimize import rosen_der
import numpy as np
import random

Now we create a function generating a fix random vector $x\in \mathbb{R}^{\rm size}$, where $x_i\in [{\rm low},{\rm high}]$.

In [ ]:
def rand(low, high, size, random_state=42, endpoint=True):
    rng = np.random.default_rng(random_state);
    return rng.integers(low, high, size=size, endpoint=endpoint);
 #rand(-1, 1, 10);   

In [ ]:
def dixon(X):
  d = len(X);
  f = (X[0] - 1) **2 + np.sum([(i+1) * (2*X[i]**2 - X[i-1])**2 for i in range(1, d)]);
  return f
def dixon_der(X):
  d=len(X);
  term1=np.zeros(d);
  term1[0]=2*(X[0]-1);
  for i in range(1,d):
    term1[i]=term1[i]+(i+1)*8*X[i]*(2*(X[i]**2)-X[i-1]);
    term1[i-1]=term1[i-1]-2*(i+1)*(2*(X[i]**2)-X[i-1]);
  return term1
def dixon_sol(d):
  x=np.zeros(d);
  x[0]=1;
  for i in range(1,d):
    x[i]=np.sqrt(0.5*x[i-1]);
  return x

Now we provide the code for steepest descent method.

In [ ]:
def STD(x_init,para,option,prob):
  print('                               Steepest descent method ')
  x=x_init;
  beta=para.beta;
  gamma=para.gamma;
  fval=prob.fval;
  grad=prob.grad;
  k=1;
  if hasattr(option, 'norm_grad'):
    norm_grad=option.norm_grad;
    while np.linalg.norm(grad(x))> norm_grad:
      if option.display==1:
        if k>1:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|Stepsize:',t);
        else:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
      g=grad(x);
      d=-g;
      t=1;
      while fval(x+t*d)>fval(x)-beta*t*(np.linalg.norm(d))**2:
        t=t*gamma;
      x=x+t*d;
      k=k+1;
    print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
  return x;

Next we introduce the code for IRG method with backtracking lineseach.

In [ ]:
def IRG_backtracking(x_init,para,option,prob):
  print('                               IRG backtracking ');
  x=x_init;
  m=len(x);
  er=rand(-1,1,m);
  error=er/np.linalg.norm(er);
  beta=para.beta;
  gamma=para.gamma;
  theta=para.theta;
  mu=para.mu;
  eps=para.eps;
  r=para.r;
  fval=prob.fval;
  grad=prob.grad;
  count=0;
  k=1;
  if hasattr(option, 'norm_grad'):
    norm_grad=option.norm_grad;
    while np.linalg.norm(grad(x))> norm_grad:
      if option.display==1:
        if k>1:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|Stepsize:',t);
        else:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
      g=grad(x)+0.5*min(eps,1/(k+1))*error;
      if np.linalg.norm(g)>r+eps:
        d=-(np.linalg.norm(g)-eps)/np.linalg.norm(g)*g;
        t=1;
        while fval(x+t*d)>fval(x)-beta*t*np.linalg.norm(d)**2:
          t=t*gamma;
        x=x+t*d;
        nulls=0;
      else:
        eps=eps*theta;
        r=r*mu;
        nulls=1;
        count=count+1;
      k=k+1;
    print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|eps:',eps,'|error:',0.5*min(eps,1/(k+1)));
  print(' IRG-backtracking non-null iterations',k-count)
  return x;

Next we introduce the code for IRG method with unbounded backtracking lineseach.

In [ ]:
def IRG_unbounded(x_init,para,option,prob):
  print('                               IRG-unbounded ');
  x=x_init;
  m=len(x);
  er=rand(-1,1,m);
  error=er/np.linalg.norm(er);
  beta=para.beta;
  gamma=para.gamma;
  theta=para.theta;
  mu=para.mu;
  eps=para.eps;
  r=para.r;
  fval=prob.fval;
  grad=prob.grad;
  count=0;
  k=1;
  if hasattr(option, 'norm_grad'):
    norm_grad=option.norm_grad;
    while np.linalg.norm(grad(x))> norm_grad:
      if option.display==1:
        if k>1:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|Stepsize:',t);
        else:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
      g=grad(x)+0.5*min(eps,1/(k+1))*error;
      if np.linalg.norm(g)>r+eps:
        d=-(np.linalg.norm(g)-eps)/np.linalg.norm(g)*g;
        t=1;
        if fval(x+d)>fval(x)-beta*np.linalg.norm(d)**2:
          while fval(x+t*d)>fval(x)-beta*t*np.linalg.norm(d)**2:
            t=t*gamma;
        else:
          while fval(x+t*d)<=fval(x)-beta*t*np.linalg.norm(d)**2:
            t=t/gamma;
          t=t*gamma;
        x=x+t*d;
        nulls=0;
      else:
        eps=eps*theta;
        r=r*mu;
        nulls=1;
        count=count+1;
      k=k+1;
    print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|eps:',eps,'|error:',0.5*min(eps,1/(k+1)));
  print(' IRG-unbounded non-null iterations',k-count)
  return x;

Next we introduce the code for RG method with unbounded backtracking lineseach.

In [ ]:
def RG_unbounded(x_init,para,option,prob):
  print('                               RG-unbounded ');
  x=x_init;
  m=len(x);
  beta=para.beta;
  gamma=para.gamma;
  theta=para.theta;
  mu=para.mu;
  eps=para.eps;
  r=para.r;
  fval=prob.fval;
  grad=prob.grad;
  count=0;
  k=1;
  if hasattr(option, 'norm_grad'):
    norm_grad=option.norm_grad;
    while np.linalg.norm(grad(x))> norm_grad:
      if option.display==1:
        if k>1:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|Stepsize:',t);
        else:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
      g=grad(x);
      if np.linalg.norm(g)>r+eps:
        d=-(np.linalg.norm(g)-eps)/np.linalg.norm(g)*g;
        t=1;
        if fval(x+d)>fval(x)-beta*np.linalg.norm(d)**2:
          while fval(x+t*d)>fval(x)-beta*t*np.linalg.norm(d)**2:
            t=t*gamma;
        else:
          while fval(x+t*d)<=fval(x)-beta*t*np.linalg.norm(d)**2:
            t=t/gamma;
          t=t*gamma;
        x=x+t*d;
        nulls=0;
      else:
        eps=eps*theta;
        r=r*mu;
        nulls=1;
        count=count+1;
      k=k+1;
    print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|eps:',eps,'|error:',0.5*min(eps,1/(k+1)));
  print(' RG-unbounded non-null iterations',k-count)
  return x;

Next we introduce the code for RG method with backtracking lineseach.

In [ ]:
def RG_backtracking(x_init,para,option,prob):
  print('                               RG backtracking ');
  x=x_init;
  m=len(x);
  beta=para.beta;
  gamma=para.gamma;
  theta=para.theta;
  mu=para.mu;
  eps=para.eps;
  r=para.r;
  fval=prob.fval;
  grad=prob.grad;
  count=0;
  k=1;
  if hasattr(option, 'norm_grad'):
    norm_grad=option.norm_grad;
    while np.linalg.norm(grad(x))> norm_grad:
      if option.display==1:
        if k>1:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|Stepsize:',t);
        else:
          print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)));
      g=grad(x);
      if np.linalg.norm(g)>r+eps:
        d=-(np.linalg.norm(g)-eps)/np.linalg.norm(g)*g;
        t=1;
        while fval(x+t*d)>fval(x)-beta*t*np.linalg.norm(d)**2:
          t=t*gamma;
        x=x+t*d;
        nulls=0;
      else:
        eps=eps*theta;
        r=r*mu;
        nulls=1;
        count=count+1;
      k=k+1;
    print(' Iter:',k,'|Value:',fval(x),'|Norm grad:',np.linalg.norm(grad(x)),'|eps:',eps,'|error:',0.5*min(eps,1/(k+1)));
  print(' RG-backtracking non-null iterations',k-count)
  return x;

Now we present numerical experiments. First we choose some linesearch parameters and running options.

In [ ]:
class para():
  beta=0.7;
  gamma=0.5;
  theta=0.7;
  mu=0.7;
  eps=5;
  r=5;
class option():
  norm_grad=0.01;
  display=0;


----------------------Testing with Rosenbrock function 20--------------------------
                               Steepest descent method 
 Iter: 4907 |Value: 9.49317533064536e-05 |Norm grad: 0.009995476197742978
                               IRG backtracking 
 Iter: 3651 |Value: 9.376640950370217e-05 |Norm grad: 0.00989642102582078 |eps: 0.003989613314880596
 IRG-backtracking non-null iterations 3631
                               RG backtracking 
 Iter: 3650 |Value: 9.408970058741499e-05 |Norm grad: 0.009937509544784198 |eps: 0.003989613314880596
 RG-backtracking non-null iterations 3630
                               IRG-unbounded 
 Iter: 3651 |Value: 9.376640950370217e-05 |Norm grad: 0.00989642102582078 |eps: 0.003989613314880596
 IRG-unbounded non-null iterations 3631
                               RG-unbounded 
 Iter: 3650 |Value: 9.408970058741499e-05 |Norm grad: 0.009937509544784198 |eps: 0.003989613314880596
 RG-unbounded non-null iterations 3630


Next we test with Rosenbrock function

In [ ]:
print('----------------------Testing with Rosenbrock function 20--------------------------');
x_init = np.zeros(20);
class prob():
  fval=rosen;
  grad=rosen_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);
print('----------------------Testing with Rosenbrock function 500--------------------------');
x_init = np.zeros(500);
class prob():
  fval=rosen;
  grad=rosen_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);
print('----------------------Testing with Rosenbrock function 2000--------------------------');
x_init = np.zeros(2000);
class prob():
  fval=rosen;
  grad=rosen_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);

----------------------Testing with Rosenbrock function 20--------------------------
                               Steepest descent method 
 Iter: 4907 |Value: 9.49317533064536e-05 |Norm grad: 0.009995476197742978
                               IRG backtracking 
 Iter: 3651 |Value: 9.376640950370217e-05 |Norm grad: 0.00989642102582078 |eps: 0.003989613314880596 |error: 0.00013691128148959474
 IRG-backtracking non-null iterations 3631
                               RG backtracking 
 Iter: 3650 |Value: 9.408970058741499e-05 |Norm grad: 0.009937509544784198 |eps: 0.003989613314880596 |error: 0.0001369487811558477
 RG-backtracking non-null iterations 3630
                               IRG-unbounded 
 Iter: 3651 |Value: 9.376640950370217e-05 |Norm grad: 0.00989642102582078 |eps: 0.003989613314880596 |error: 0.00013691128148959474
 IRG-unbounded non-null iterations 3631
                               RG-unbounded 
 Iter: 3650 |Value: 9.408970058741499e-05 |Norm grad: 0.009937509544784198 |e

Next we test with Dixon & Price function

In [ ]:
print('----------------------Testing with Dixon&Price function 20--------------------------');
x_init = np.ones(20);
class prob():
  fval=dixon;
  grad=dixon_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);
print('----------------------Testing with Dixon&Price function 500--------------------------');
x_init = np.ones(500);
class prob():
  fval=dixon;
  grad=dixon_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);
print('----------------------Testing with Dixon&Price function 2000--------------------------');
x_init = np.ones(2000);
class prob():
  fval=dixon;
  grad=dixon_der;
STD(x_init,para,option,prob);
IRG_backtracking(x_init,para,option,prob);
RG_backtracking(x_init,para,option,prob);
IRG_unbounded(x_init,para,option,prob);
RG_unbounded(x_init,para,option,prob);

----------------------Testing with Dixon&Price function 20--------------------------
                               Steepest descent method 
 Iter: 1928 |Value: 2.7790504935867027e-05 |Norm grad: 0.009946903078270647
                               IRG backtracking 
 Iter: 1004 |Value: 2.710264941790442e-05 |Norm grad: 0.009959857961855707 |eps: 0.003989613314880596
 IRG-backtracking non-null iterations 984
                               RG backtracking 
 Iter: 998 |Value: 2.7556184775233842e-05 |Norm grad: 0.009890570748953052 |eps: 0.003989613314880596
 RG-backtracking non-null iterations 978
                               IRG-unbounded 
 Iter: 1004 |Value: 2.710264941790442e-05 |Norm grad: 0.009959857961855707 |eps: 0.003989613314880596
 IRG-unbounded non-null iterations 984
                               RG-unbounded 
 Iter: 998 |Value: 2.7556184775233842e-05 |Norm grad: 0.009890570748953052 |eps: 0.003989613314880596
 RG-unbounded non-null iterations 978
